In [4]:
!pip install -U transformers
!pip install -U datasets
!pip install tensorboard
!pip install sentencepiece
!pip install accelerate
!pip install evaluate
!pip install rouge_score

/bin/bash: /home/mohan/miniconda3/envs/env/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
/bin/bash: /home/mohan/miniconda3/envs/env/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
/bin/bash: /home/mohan/miniconda3/envs/env/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
/bin/bash: /home/mohan/miniconda3/envs/env/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
/bin/bash: /home/mohan/miniconda3/envs/env/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
/bin/bash: /home/mohan/miniconda3/envs/e

In [1]:
from transformers import BartTokenizer, BartForConditionalGeneration
 
import glob
import pprint
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

 
pp = pprint.PrettyPrinter()

In [2]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

In [3]:
def summarize_text(text, model, tokenizer, max_length=512, num_beams=5):
    # Preprocess the text
    inputs = tokenizer.encode(
        "summarize: " + text,
        return_tensors='pt',
        max_length=max_length,
        truncation=True
    )
 
    # Generate the summary
    summary_ids = model.generate(
        inputs,
        max_length=50,
        num_beams=num_beams,
        # early_stopping=True,
    )
 
    # Decode and return the summary
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [6]:
import torch
import pprint
import evaluate
import numpy as np
 
from transformers import (
    BartTokenizer,
    BartForConditionalGeneration,
    TrainingArguments,
    Trainer
)


In [7]:
dataset_train = pd.read_csv('/home/mohan/infy/data/fined/split/train.csv')
dataset_valid = pd.read_csv('/home/mohan/infy/data/fined/split/validation.csv')

df1 = pd.read_csv('/home/mohan/infy/data/fined/split/train.csv')
dataset1 = ds.dataset(pa.Table.from_pandas(df1).to_batches())

### convert to Huggingface dataset
dataset_train = Dataset(pa.Table.from_pandas(df1))
print(dataset_train.__class__)

df2 = pd.read_csv('/home/mohan/infy/data/fined/split/validation.csv')
dataset1 = ds.dataset(pa.Table.from_pandas(df2).to_batches())
dataset_valid = Dataset(pa.Table.from_pandas(df2))
print(dataset_valid.__class__)

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


In [8]:
print(dataset_train)
print(dataset_valid)

Dataset({
    features: ['text', 'summary'],
    num_rows: 50165
})
Dataset({
    features: ['text', 'summary'],
    num_rows: 6271
})


In [9]:
MODEL = 'facebook/bart-large'
BATCH_SIZE = 4
NUM_PROCS = 4
EPOCHS = 10
OUT_DIR = '/home/mohan/infy/models/fine_tuned_Text_Summ'
MAX_LENGTH = 512

In [10]:
tokenizer = BartTokenizer.from_pretrained(MODEL)
 
# Function to convert text data into model inputs and targets
def preprocess_function(examples):
    inputs = [f"summarize: {article}" for article in examples['text']]
    model_inputs = tokenizer(
        inputs,
        max_length=MAX_LENGTH,
        truncation=True,
        padding='max_length'
    )
 
    # Set up the tokenizer for targets
    targets = [summary for summary in examples['summary']]
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=MAX_LENGTH,
            truncation=True,
            padding='max_length'
        )
 
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
 
# Apply the function to the whole dataset
tokenized_train = dataset_train.map(
    preprocess_function,
    batched=True,
    num_proc=NUM_PROCS
)
tokenized_valid = dataset_valid.map(
    preprocess_function,
    batched=True,
    num_proc=NUM_PROCS
)

Map (num_proc=4):   0%|          | 0/50165 [00:00<?, ? examples/s]

/home/mohan/miniconda3/envs/infosys/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/mohan/miniconda3/envs/infosys/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/mohan/miniconda3/envs/infosys/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is d

Map (num_proc=4):   0%|          | 0/6271 [00:00<?, ? examples/s]

/home/mohan/miniconda3/envs/infosys/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/mohan/miniconda3/envs/infosys/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/mohan/miniconda3/envs/infosys/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is d

In [11]:
model = BartForConditionalGeneration.from_pretrained(MODEL)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

406,291,456 total parameters.
406,291,456 training parameters.


In [12]:
rouge = evaluate.load("rouge")
 
def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions[0], eval_pred.label_ids
 
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
 
    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
        rouge_types=[
            'rouge1',
            'rouge2',
            'rougeL'
        ]
    )
 
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
 
    return {k: round(v, 4) for k, v in result.items()}

In [13]:
def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

In [14]:
training_args = TrainingArguments(
    output_dir=OUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=OUT_DIR,
    logging_steps=10,
    evaluation_strategy='steps',
    eval_steps=2000,
    save_strategy='epoch',
    save_total_limit=2,
    report_to='tensorboard',
    learning_rate=0.0001,
    dataloader_num_workers=4
)
 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics
)
 
history = trainer.train()

/home/mohan/miniconda3/envs/infosys/lib/python3.11/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Gen Len
2000,0.563000,0.586864,0.584700,0.292700,0.507500,124.277900
4000,0.463400,0.573142,0.590800,0.299500,0.515600,124.277900
6000,0.533200,0.558139,0.591500,0.301600,0.516900,124.277900
8000,0.601300,0.555438,0.588900,0.296500,0.513600,124.277900
10000,0.723500,0.547032,0.592400,0.300100,0.518100,124.277900
12000,0.539100,0.581864,0.540800,0.241900,0.461700,124.277900
14000,0.668100,0.536584,0.599300,0.309200,0.524700,124.277900
16000,0.443000,0.533588,0.600000,0.307900,0.526000,124.277900
18000,0.408200,0.553599,0.582600,0.287800,0.506800,124.277900
20000,0.567400,0.526812,0.602000,0.313800,0.528700,124.277900


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

In [15]:
model.save_pretrained("/home/mohan/infy/models/fine_tuned_Text_Summ/saved")
tokenizer.save_pretrained("/home/mohan/infy/models/fine_tuned_Text_Summ/saved")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('/home/mohan/infy/models/fine_tuned_Text_Summ/saved/tokenizer_config.json',
 '/home/mohan/infy/models/fine_tuned_Text_Summ/saved/special_tokens_map.json',
 '/home/mohan/infy/models/fine_tuned_Text_Summ/saved/vocab.json',
 '/home/mohan/infy/models/fine_tuned_Text_Summ/saved/merges.txt',
 '/home/mohan/infy/models/fine_tuned_Text_Summ/saved/added_tokens.json')